In [0]:
%run /Workspace/Users/249y1a0232@ksrmce.ac.in/AzureDataEngineering-DataBricks/_resources/setup_auth

In [0]:
from pyspark.sql.functions import col, regexp_replace, when

# 1. Read from Bronze
path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/bronze"

silver_data = spark.read.format('delta').load(path)
display(silver_data)

gold_df = (silver_data.groupBy("addr_state")
           .agg({"is_default": "avg", "loan_amnt": "sum"})
           .withColumnRenamed("avg(is_default)", "default_rate")
           .withColumnRenamed("sum(loan_amnt)", "total_loan_volume"))


gold_df.show()
# 3. Save for Power BI / Reporting
gold_df.write.format("delta").mode("overwrite").save(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/gold/")